## **1. 라이브러리 설치**

In [ ]:
!pip install ultralytics
import ultralytics

import os
import shutil
import yaml
import glob
import time
from ultralytics import YOLO
from google.colab import drive
from tqdm.auto import tqdm

📦 Ultralytics 라이브러리 설치 중...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## **2. 데이터셋 경로 설정**

In [ ]:
drive.mount('/content/drive')

# 구글 드라이브 경로 (훈련 데이터셋, 검증 데이터셋)
DRIVE_TRAIN_ROOT = "/content/drive/MyDrive/Capstone_YOLO_Training_Dataset_10k"
DRIVE_VAL_ROOT   = "/content/drive/MyDrive/Capstone_YOLO_Validation_Dataset_3k"

# Colab 로컬 경로
LOCAL_TRAIN_ROOT = "/content/Capstone_YOLO_Training_Dataset"
LOCAL_VAL_ROOT   = "/content/Capstone_YOLO_Validation_Dataset"

PROJECT_SAVE_DIR = "/content/drive/MyDrive/YOLO_Result"
RUN_NAME = "yolo11n_person_retry"

# 훈련 파라미터
EPOCHS = 100
BATCH_SIZE = 128  # A100/L4 GPU를 사용하기 때문에 128 선택
IMG_SIZE = 640
WORKERS = 4       # Colab CPU 병목 방지

Mounted at /content/drive


## **3. 함수 선언**

In [ ]:

# 데이터셋 카운트 함수
def count_files(directory):
    """폴더 내 파일 개수 카운트"""
    if not os.path.exists(directory): return 0
    return len(glob.glob(os.path.join(directory, "**", "*.*"), recursive=True))

# 데이터셋 카피 함수
def copy_with_progress(src, dst, description="복사 중"):

    # 1. 소스 파일 리스트업
    files = glob.glob(os.path.join(src, "**", "*.*"), recursive=True)
    total_files = len(files)

    if total_files == 0:
        print(f"🚨 [오류] {src} 경로에 파일이 없습니다.")
        return False

    # 2. 기존 폴더 정리
    if os.path.exists(dst):
        shutil.rmtree(dst)

    print(f"📦 [{description}] 총 {total_files}개 파일 복사 시작...")

    # 3. 복사 루프 (진행률 표시)
    try:
        for file_path in tqdm(files, desc=description, unit="file"):
            # 원본 경로에서 상대 경로 추출 (예: images/train/1.jpg)
            rel_path = os.path.relpath(file_path, src)
            dest_path = os.path.join(dst, rel_path)

            # 대상 폴더가 없으면 생성
            os.makedirs(os.path.dirname(dest_path), exist_ok=True)

            # 파일 복사
            shutil.copy2(file_path, dest_path)

    except Exception as e:
        print(f"\n❌ 복사 중 오류 발생: {e}")
        return False

    print(f"✅ [{description}] 복사 완료!\n")
    return True

# yaml 생성 함수
def create_yaml(train_root, val_root):
    # 경로 자동 감지
    train_img = os.path.join(train_root, "images", "train")
    if not os.path.exists(train_img): train_img = os.path.join(train_root, "train", "images")

    val_img = os.path.join(val_root, "images", "val")
    if not os.path.exists(val_img): val_img = train_img

    # 최종 파일 검증
    jpg_count = len(glob.glob(os.path.join(train_img, "*.jpg"))) + \
                len(glob.glob(os.path.join(train_img, "*.JPG")))

    print(f"🧐 데이터셋 최종 점검: {train_img}")
    print(f"   👉 감지된 이미지 파일: {jpg_count}장")

    if jpg_count == 0:
        print("🚨 [심각] 이미지 파일이 없습니다. 복사가 실패했거나 경로가 잘못되었습니다.")
        return None

    data = {
        'train': train_img,
        'val': val_img,
        'nc': 1,
        'names': ['person']
    }

    yaml_path = os.path.join(train_root, "data_progress.yaml")
    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)

    return yaml_path

## **4. 메인 함수 (훈련)**

In [ ]:
def main():
    print("🚀 YOLO 학습 준비 (실시간 진행률 표시 모드)")

    # 1. 훈련셋 복사 (프로그레스 바 표시)
    if not copy_with_progress(DRIVE_TRAIN_ROOT, LOCAL_TRAIN_ROOT, "훈련 데이터"):
        return

    # 2. 검증셋 복사
    if os.path.exists(DRIVE_VAL_ROOT):
        copy_with_progress(DRIVE_VAL_ROOT, LOCAL_VAL_ROOT, "검증 데이터")
    else:
        print("⚠️ 검증셋 경로가 없어 훈련셋을 검증용으로 사용합니다.")

    # 3. YAML 생성
    yaml_path = create_yaml(LOCAL_TRAIN_ROOT, LOCAL_VAL_ROOT)
    if not yaml_path: return

    # 4. 학습 시작
    print("\n🔥 YOLOv11 학습 시작...")
    model = YOLO('yolo11n.pt')
    model.train(
        data=yaml_path,
        epochs=100,
        imgsz=640,
        batch=128,
        project=PROJECT_SAVE_DIR,
        name=RUN_NAME,
        workers=4,
        exist_ok=True
    )

if __name__ == "__main__":
    main()

🚀 YOLO 학습 준비 (실시간 진행률 표시 모드)
📦 [훈련 데이터] 총 20086개 파일 복사 시작...


훈련 데이터:   0%|          | 0/20086 [00:00<?, ?file/s]

✅ [훈련 데이터] 복사 완료!

📦 [검증 데이터] 총 6000개 파일 복사 시작...


검증 데이터:   0%|          | 0/6000 [00:00<?, ?file/s]

✅ [검증 데이터] 복사 완료!

🧐 데이터셋 최종 점검: /content/Capstone_YOLO_Training_Dataset/images/train
   👉 감지된 이미지 파일: 10043장

🔥 YOLOv11 학습 시작...
Ultralytics 8.3.230 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Capstone_YOLO_Training_Dataset/data_progress.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, 